# Simulator
### Simple version 

In [36]:
using Pkg

Pkg.add("Pipe")

using Pipe


  Resolving package versions...
No Changes to `C:\Users\Owner\.julia\environments\v1.5\Project.toml`
No Changes to `C:\Users\Owner\.julia\environments\v1.5\Manifest.toml`


### Data structures and types

Abstract types of general entities in the simulator :

**Node** - vertex of the Graph that represents a page with content. 

**Graph** - directed graph , it represents page linking. It consists of Nodes. If there is an edge from node X to Y , there is a direct link on corresponding page X to page Y .

**User** - represents the user that visit the site ( set of pages ). 


In [2]:
abstract type Graph end

abstract type Node end

abstract type User end



The simplest instantiation of general objects.  

In [3]:
struct Graph1 <: Graph
    graph :: Dict{Node,Set{Node}}  # Key - Node , Value - Set of its neighbors
end

struct Node1 <: Node
    id :: String                   # Unique id of the node   
    NameInInput :: Any             # The name of the node as it was given in the input 
    isFinal :: Bool                # Whether the node is a final node ( the user leaved the pages )
end

struct User1 <: User
   id :: String                    # Unique id of the user
   choiseFunction                  # The function that defines the behaviour of the user when she traverses the graph 
end

### Auxiliary functions

In [33]:
# Create a counter , used to produce unique ids
function id_generator()
    id = 0 
    return ()-> id = id + 1
end

nodeIdGenerator = id_generator()    # Nodes id generator
userIdGenerator = id_generator()    # Users id generator



# Return Node that corresponds to some name (that was given in input source , typicaly number)

function get_node(NameInInput, graph:: Graph1)
    nodes = graph.graph |> keys |> collect
    index = findfirst(node -> NameInInput == node.NameInInput,nodes) 
   return  nodes[index]
    end

# Whether the node is a final node ( the user leaved)
function is_terminal(node :: Node1)
    return node.isFinal 

end

# Given array of nodes , print their names devided by "=>"
# It is supposed to print the users path in the graph
function printPath(path :: Array{Node,1})
    return @pipe map(node-> node.NameInInput,path) |> join(_,"=>")
    
end;

### Producing the graph 

Instantiate the graph given array of pairs , each pair represents edge in graph (source, destination)

In [32]:
function createGraph(simple_tuples :: Array{Tuple{Int64,Int64},1})
    
    node_dict = Dict()         # Dict from number to corresponding node
    graph = Graph1(Dict())     # Resulting graph
    
    
    # For each pair of numbers in the input , create corresponding nodes , if they don't exist, 
    # and store node node2 in the set of node1's neighbors
    for (n1,n2) in simple_tuples
           
        node1 = get!(()->(@pipe nodeIdGenerator() |> string |> Node1(_,n1,false)),node_dict,n1)
        
        node2 =  get!(()->(@pipe nodeIdGenerator() |> string |> Node1(_,n2,false)),node_dict,n2)
        
        neighbors = get!(Set,graph.graph,node1)
        
        push!(neighbors,node2)      
        
    end
    
    # Create terminal node and lings from all nodes in graph to it
    terminalNode = @pipe nodeIdGenerator() |> string |> Node1(_,"exit",true)

    foreach(x->push!(x,terminalNode),values(graph.graph))

  return graph  
    
    end;

### Main interface functions

This functions will be extended when more complex user behavior will be implemented 

In [35]:
# Given node and graph , return node's neighbors
function getNeibhors(node :: Node, graph :: Graph1) ::  Set{Node}
        return graph.graph[node]
end


# Given set of nodes , the user choose one node .
# The function is supposed to be used for defining the next node to traverse to .
# The inner choice function of the user is called.
function chooseNextNode(user :: User1, nodes :: Set{Node}) :: Node
    user.choiseFunction(nodes)
 end;






## Main function 

Given the user the graph and the initial node , run the simulation of user traversing the graph according to her inner policy.
Return the summary of user's traversal  ( currently only a path )

In [41]:
function performWalk(user :: User, graph :: Graph, initialNode :: Node)
    currentNode = initialNode
    path = Node[]               # The nodes the user visited
    
    while !is_terminal(currentNode)
       push!(path,currentNode)
        
       neighbours = getNeibhors(currentNode, graph)
       currentNode = chooseNextNode(user, neighbours)
        
    end

    return Dict("path" => path)
    end;

### The example of run 

The user has a simple policy , choose next node randomly.
Initial node is node 1 .


In [40]:
user = User1("1",x -> rand(x))   # The second arg is user's policy of choosing the node 

graph = createGraph([(1,2),(2,3),(1,3),(3,1)])

initialNode = get_node(1,graph)       

result = performWalk(user,graph,initialNode)   # Simulation 

printPath(result["path"])

"1=>3=>1=>2=>3=>1=>2"